## Import

In [21]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.feature_selection import f_classif

from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

from IPython.display import display
pd.set_option('display.max_columns', None)

from sklearn.neural_network import MLPRegressor

In [2]:
df = pd.read_csv('data/Cleaned_amzn_df.csv',index_col='AMZN')
df.dropna(inplace=True)

In [3]:
df = df.iloc[-500:]
df

,Open,High,Low,Close,Volume,sentiment,next_day_close_percent,percent_increase,percent_increase_close,four_percent,SMA,SMA_diff,Stochastic,Stochastic_diff,rsi_up_var,rsi_down_var,RSI,RSI_diff,ROC,ROC_diff,tr,ATR,ATR_diff,pos_dx,neg_dx,dx,ADX,ADX_diff
AMZN,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-02-13,1647.00,1656.38,1637.110,1640.00,3560321.0,Sell,-1.057927,-0.128049,-0.757317,0,1625.578889,-8.747778,43.171235,1.173003,1.99,0.00,48.393345,-1.396206,-4.580708,-2.639890,19.27,47.920000,-0.443571,0.00,38.23,25.018348,29.017135,-0.391278
2019-02-14,1624.50,1637.90,1606.060,1622.65,4120524.0,Sell,-0.905925,0.757403,0.549102,0,1625.181111,-0.397778,32.944297,-10.226938,0.00,17.35,44.862118,-3.531227,-0.220141,4.360567,33.94,48.305000,0.385000,0.00,0.00,41.299223,27.570711,-1.446424
2019-02-15,1627.86,1628.91,1604.500,1607.95,4343893.0,Buy,1.220809,2.901210,1.557884,0,1622.363333,-2.817778,24.279399,-8.664898,0.00,14.70,46.661165,1.799047,-1.552675,-1.332534,24.41,46.014286,-2.290714,0.00,0.00,41.299223,26.966062,-0.604649
2019-02-19,1601.00,1634.00,1600.560,1627.58,3681656.0,Sell,-0.336696,1.660133,0.541909,0,1618.893333,-3.470000,35.850280,11.570881,19.63,0.00,53.974244,7.313079,-1.882675,-0.330000,33.44,45.033571,-0.980714,5.09,0.00,42.575272,24.074783,-2.891279
2019-02-20,1630.00,1634.93,1610.120,1622.10,3337589.0,Sell,-0.163985,2.003576,1.170705,0,1616.875556,-2.017778,32.620100,-3.230180,0.00,5.48,43.152645,-10.821599,-1.107142,0.775533,24.81,40.872143,-4.161429,0.00,9.56,22.878377,22.709063,-1.365721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-01,3242.36,3350.26,3235.025,3342.88,4160212.0,Buy,1.110420,2.725793,1.110420,0,3278.001111,24.680000,92.439454,49.184929,136.68,0.00,68.107886,16.021667,7.117497,3.833290,144.06,81.959714,5.097143,113.27,0.00,32.956101,18.797755,0.513129
2021-02-02,3380.00,3427.74,3361.125,3380.00,6183716.0,Sell,-1.996154,1.597633,-0.823964,0,3290.958889,12.957778,85.645127,-6.794327,37.12,0.00,69.577435,1.469549,3.573595,-3.543902,84.86,84.011014,2.051300,0.00,126.10,7.195686,17.470854,-1.326901
2021-02-03,3425.01,3434.00,3308.620,3312.53,7088781.0,Buy,0.557580,1.946247,1.196065,0,3291.574444,0.615556,64.150164,-21.494964,0.00,67.47,60.714286,-8.863149,0.167524,-3.406072,125.38,88.029586,4.018571,6.26,0.00,0.867212,17.325803,-0.145051


## Train/Test Split

In [4]:
y = df['next_day_close_percent']
x = df[['SMA', 'SMA_diff', 'Stochastic', 'Stochastic_diff', 'RSI', 'RSI_diff', 'ROC', 'ROC_diff', 'ATR', 'ATR_diff', 'ADX', 'ADX_diff']]
x = df[['SMA', 'Stochastic', 'RSI', 'ROC', 'ATR', 'ADX']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 123)

In [5]:
mm_scaler = MinMaxScaler()
mm_scaler.fit(x_train)

x_train = mm_scaler.transform(x_train)
x_test = mm_scaler.transform(x_test)

In [6]:
# scaler = StandardScaler()

# scaled_x_train = scaler.fit_transform(x_train)
# scaled_y_train = scaler.fit_transform(y_train)

# scaled_x_test = scaler.fit_transform(x_test)
# scaled_y_test = scaler.fit_transform(y_test)

## Modeling

### Function

In [7]:
def rmse(model, xtrain, ytrain, xtest, ytest):
    yhat_train = model.predict(xtrain)
    rmse_train = np.sqrt(metrics.mean_squared_error(ytrain, yhat_train))

    yhat_test = model.predict(xtest)
    rmse_test = np.sqrt(metrics.mean_squared_error(ytest, yhat_test))

    print('RMSE Train:', rmse_train)
    print('RMSE Test:', rmse_test)

### Decision Tree Regression

In [18]:
dtr = DecisionTreeRegressor(criterion='mse',
                            max_depth=2,
                            max_features='log2',
                            min_samples_leaf=3,
                            min_samples_split=5)
dtr.fit(x_train, y_train)

rmse(dtr, x_train, y_train, x_test, y_test)

RMSE Train: 1.9931852551398375
RMSE Test: 1.6035571605074477


### SVR

In [19]:
svr = SVR(C=.001, degree=1, gamma=0.001, kernel='rbf')
# svr.fit(scaled_x_train, y_train)
svr.fit(x_train, y_train)
# rmse(svr, scaled_x_train, y_train, scaled_x_test, y_test)
rmse(svr, x_train, y_train, x_test, y_test)

RMSE Train: 2.034103183144371
RMSE Test: 1.6226854725252058


### KNR

In [20]:
knr = KNeighborsRegressor(algorithm='auto', metric='manhattan', n_neighbors=19, weights='distance')
# knr.fit(scaled_x_train, y_train)
# rmse(knr, scaled_x_train, y_train, scaled_x_test, y_test)

knr.fit(x_train, y_train)
rmse(knr, x_train, y_train, x_test, y_test)

RMSE Train: 0.0
RMSE Test: 1.6109122303906156


### Random Forest Regressor

In [13]:
rfr = RandomForestRegressor(criterion='mae', max_depth=30, max_features='sqrt', min_samples_leaf=10, min_samples_split=5, n_estimators=20)
rfr.fit(x_train, y_train)
rmse(rfr, x_train, y_train, x_test, y_test)

RMSE Train: 1.8457042672406263
RMSE Test: 1.6235573843114346


### AdaBoost Regressor

In [14]:
ada_reg = AdaBoostRegressor(learning_rate=.05, loss='square', n_estimators=100)
ada_reg.fit(x_train, y_train)
rmse(ada_reg, x_train, y_train, x_test, y_test)

RMSE Train: 1.8083236069738058
RMSE Test: 1.6206420238838988


In [15]:
ada_reg.score(x_train, y_train)

0.20940925085472895

In [16]:
ada_reg.score(x_test, y_test)

0.0016858455533186323

###  Gradient Boosting Regressor

In [17]:
grad_boost_reg = GradientBoostingRegressor(criterion= 'mse', learning_rate= 0.001, loss= 'huber', n_estimators = 150)
grad_boost_reg.fit(x_train, y_train)
rmse(grad_boost_reg, x_train, y_train, x_test, y_test)

RMSE Train: 1.9982113003746613
RMSE Test: 1.6064425158019298


### MLP Regressor

In [50]:
nn = MLPRegressor(hidden_layer_sizes=(300,100,), activation='tanh', max_iter=1000, solver='adam')
nn.fit(x_train, y_train)
rmse(nn, x_train, y_train, x_test, y_test)

RMSE Train: 2.0310215095908437
RMSE Test: 1.6299344067374673


### Test

In [ ]:
a = SVR()

In [ ]:
y